In [ ]:
# importing packages
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, Normalizer
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, roc_curve, auc
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold

In [ ]:
print(os.getcwd())

In [ ]:
slash = "/" if "Linux" in platform.platform() else "\\"

In [ ]:
data_link = os.getcwd() + f'{slash}Dataset_TBI.xlsx'

In [ ]:
df_data = pd.read_excel(data_link, index_col=0)
print(df_data)

In [ ]:
df_data.isnull()

In [ ]:
df_data.describe()

In [ ]:
df_data.info()

In [ ]:
df_data['GOS-E (t2)'].value_counts()

In [ ]:
df_tbi = df_data

In [ ]:
old_value1 = [1, 2, 3 ,4]
old_value2 = [5, 6, 7, 8]
new_value1 = 0
new_value2 = 1
for i, j in zip(old_value1, old_value2):
    df_tbi['GOS-E (t2)'] = df_tbi['GOS-E (t2)'].replace(i, new_value1)
    df_tbi['GOS-E (t2)'] = df_tbi['GOS-E (t2)'].replace(j, new_value2)

In [ ]:
df_tbi['GOS-E (t2)'].value_counts()

In [ ]:
drop_list = ['Diagnosis at Discharge (t2)', 'GOS-E (t1)', 'CRS-R (t2)', 'RLAS (t2)', 'DRS (t2)', 'ERBI A (t2)', 'ERBI B (t2)']
for i in range(7):
    df_tbi = df_tbi.drop(drop_list[i], axis = 1)
# df_tbi = df_tbi.drop('GOS-E (t1)', axis = 1)
# df_tbi = df_tbi.drop('Diagnosis at Discharge (t2)', axis = 1)
# df_tbi = df_tbi.drop('Diagnosis at Discharge (t2)', axis = 1)

In [ ]:
target = df_tbi['GOS-E (t2)']
features = df_tbi.drop('GOS-E (t2)', axis = 1)

## Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
encoder1 = OneHotEncoder()
encoder2 = LabelEncoder()

In [ ]:
df_tbi['Marshall (t0)'] = encoder2.fit_transform(df_tbi['Marshall (t0)'])
df_tbi['Sex'] = encoder2.fit_transform(df_tbi['Sex'])
df_tbi['Entry Diagnosis (t0)'] = encoder2.fit_transform(df_tbi['Entry Diagnosis (t0)'])

# Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
features_array = scaler.fit_transform(features)
features_scaled = pd.DataFrame(features_array)
features_scaled

# Imputer

In [ ]:
# Tạo đối tượng KNNImputer với tham số n_neighbors=25
knn_imputer = KNNImputer(n_neighbors=25)
df = knn_imputer.fit_transform(features_scaled)
df = pd.DataFrame(features_scaled)
df

## MRMR

In [ ]:
import mrmr
# from mrmr import mrmr_classif
import pymrmr
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mutual_info_score

In [ ]:
def mrmr(X, y):
    num_features = X.shape[1]
    selected_features = []
    mrmr_scores = []

    for _ in range(num_features):
        scores = []

        for feature in range(num_features):
            if feature not in selected_features:
                relevance = mutual_info_classif(X.iloc[:, feature].astype(float).values.reshape(-1, 1), y.astype(float))
                redundancy = sum([mutual_info_score(X.iloc[:, feature].astype(float), X.iloc[:, selected_feature].astype(float)) for selected_feature in selected_features])
                mrmr_score = relevance - (1 / len(selected_features) if len(selected_features) > 0 else 0) * redundancy
                scores.append((feature, mrmr_score))
                mrmr_scores.append(mrmr_score)

        # scores = sorted(scores, key = lambda x: x[1], reverse = True)
        # selected_feature = scores[0][0]
        # selected_features.append(selected_feature)

    # return selected_features, mrmr_scores, scores
    return scores
    
# Example usage with your data
scores = mrmr(X= df, y=target)
# selected_features, mrmr_scores, scores = mrmr(X= df_new_tbi, y=df_new_tbi_target)

print(scores)
# print("Selected Features:", selected_features)
# for feature, score in zip(selected_features, mrmr_scores):
#     print(f"Feature {feature}: MRMR Score = {score}")

In [ ]:
# l = {'Age':0.10352305, 'Sex':0.0072446, 'Marshall (t0)': 0.05729801, 'Entry Diagnosis (t0)':0.16496111, 'CRS-R (t1)':0.18392238,
#        'RLAS (t1)':0.14162986, 'DRS (t1)':0.14655443, 'ERBI A (t1)':0.0766897, 'ERBI B (t1)':0.03195924}

# l_sorted = sorted(l.items(), key=lambda x:x[1], reverse=True)
# print(l_sorted)
# l_sorted_dict = {'CRS-R (t1)': 0.18392238, 'Entry Diagnosis (t0)': 0.16496111, 'DRS (t1)': 0.14655443, 
#             'RLAS (t1)': 0.14162986, 'Age': 0.10352305, 'ERBI A (t1)': 0.0766897, 'Marshall (t0)': 0.05729801, 
#             'ERBI B (t1)': 0.03195924, 'Sex': 0.0072446}

# names = list(l_sorted_dict.keys())
# values = list(l_sorted_dict.values())
# plt.figure(figsize=(14,6))
# plt.bar(range(len(l_sorted_dict)), values, tick_label=names)
# plt.show()

#### RLAS (t1) Age DRS (t1) CRS-R (t1)	Entry Diagnosis (t0) ERBI A (t1) Marshall (t0) ERBI B (t1) Sex

## Spliting set

In [ ]:
X_selected = df[['CRS-R (t1)', 'Entry Diagnosis (t0)', 'DRS (t1)','RLAS (t1)', 'Age']]
X_selected
y = target

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2)

### Model training

#### LOOCV

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import make_scorer, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


loocv = LeaveOneOut()
# Define custom scoring function
def custom_scoring(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return {
        'accuracy': acc,
        'f1_score': f1,
        'sensitivity': recall,
        'specificity': specificity
    }

# Define classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'SVM': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'Gaussian Naive Bayes': GaussianNB(),
    'Random Forest': RandomForestClassifier()
}

# Define parameter grids for each classifier
param_grids = {
    'Logistic Regression': {'C': [0.1, 1, 10, 100], 'penalty': ['l2']},
    'SVM': {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']},
    'Decision Tree': {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]},
    'KNN': {'n_neighbors': [3, 5, 7, 9]},
    'Gaussian Naive Bayes': {},
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
}

# Perform GridSearchCV for each classifier
for classifier_name, classifier in classifiers.items():
    print("Classifier:", classifier_name)
    
    # Perform GridSearchCV
    grid_search = GridSearchCV(estimator=classifier, param_grid=param_grids[classifier_name], 
                               scoring=make_scorer(custom_scoring), cv=loocv, refit='accuracy', verbose=1)
    
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    
    # Get the best parameters
    best_params = grid_search.best_params_
    print("Best Parameters:", best_params)
    
    # Get the best estimator
    best_model = grid_search.best_estimator_
    
    # Evaluate the best model
    y_pred_best = cross_val_predict(best_model, X_train, y_train, cv=loocv)
    scores_best = custom_scoring(y_train, y_pred_best)
    print("Best Model Scores:", scores_best)


### k-fold

In [ ]:
# Define 10-fold cross-validation
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)
# Define custom scoring function
def custom_scoring(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return {
        'accuracy': acc,
        'f1_score': f1,
        'sensitivity': recall,
        'specificity': specificity
    }

# Define classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'SVM': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'Gaussian Naive Bayes': GaussianNB(),
    'Random Forest': RandomForestClassifier()
}

# Define parameter grids for each classifier
param_grids = {
    'Logistic Regression': {'C': [0.1, 1, 10, 100], 'penalty': ['l2']},
    'SVM': {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']},
    'Decision Tree': {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]},
    'KNN': {'n_neighbors': [3, 5, 7, 9]},
    'Gaussian Naive Bayes': {},
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
}

# Perform GridSearchCV for each classifier
for classifier_name, classifier in classifiers.items():
    print("Classifier:", classifier_name)
    
    # Perform GridSearchCV
    grid_search = GridSearchCV(estimator=classifier, param_grid=param_grids[classifier_name], 
                               scoring=make_scorer(custom_scoring), cv=k_fold, refit='accuracy', verbose=1)
    
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    
    # Get the best parameters
    best_params = grid_search.best_params_
    print("Best Parameters:", best_params)
    
    # Get the best estimator
    best_model = grid_search.best_estimator_
    
    # Evaluate the best model
    y_pred_best = cross_val_predict(best_model, X_train, y_train, cv = k_fold)
    scores_best = custom_scoring(y_train, y_pred_best)
    print("Best Model Scores:", scores_best)
